# bucket to sql


In [1]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=f5YPdaaJrOP2tZHkzwvoYFe8L91qmk&access_type=offline&code_challenge=bsHms5H3yodEVk_GJ2Q_Y3N6De_g-fY-FzR-oxRtnHQ&code_challenge_method=S256


You are now logged in as [bharjihed@gmail.com].
Your current project is [projet-gcp-450616].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
# Installation des dépendances nécessaires
%pip install -q pandas google-cloud-storage langchain-google-vertexai langchain-google-cloud-sql-pg python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import pandas as pd
from google.cloud import storage
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_pg import PostgresEngine, PostgresVectorStore
from langchain_core.documents.base import Document
from dotenv import load_dotenv
import asyncio

In [4]:
PROJECT_ID = "projet-gcp-450616"  # À modifier
BUCKET_NAME = "soraka_bucket"      # À modifier
DATABASE = "health_db"       # À modifier
INSTANCE = "soraka-instance"       # À modifier
DB_USER = "postgres"           # À modifier

# Chargement des variables d'environnement pour les informations sensibles
load_dotenv()
DB_PASSWORD = os.environ["DB_PASSWORD"]

In [5]:


# Initialize a client
client = storage.Client()

bucket = client.get_bucket(BUCKET_NAME)

# List all files in the 'data' directory
blobs = bucket.list_blobs(prefix="data/")

print("Files in 'dauphine-bucket/data':")
for blob in blobs:
    print(blob.name)

Files in 'dauphine-bucket/data':
data/
data/medquad.csv


In [6]:
# Specify the file path (replace with an actual file from your bucket)
file_path = 'data/medquad.csv'

# Get the blob object
blob = bucket.get_blob(file_path)


if blob:
    print(f"Information for '{file_path}':")
    print(f"Size: {blob.size} bytes")
    print(f"Content Type: {blob.content_type}")
    updated_on = blob.updated
    formatted_updated_on = updated_on.strftime("%Y-%m-%d %H:%M:%S")  # Format: YYYY-MM-DD HH:MM:SS
    print(f"Updated On: {formatted_updated_on}")
    print(f"Blob name: {blob.name}")
else:
    print(f"File '{file_path}' not found in the bucket.")

Information for 'data/medquad.csv':
Size: 22835609 bytes
Content Type: text/csv
Updated On: 2025-02-11 19:18:07
Blob name: data/medquad.csv


In [7]:
DOWNLOADED_LOCAL_DIRECTORY = "./downloaded_files"

# S'assurer que le dossier downloaded_files existe
os.makedirs(DOWNLOADED_LOCAL_DIRECTORY, exist_ok=True)

# Initialiser le client Storage
client = storage.Client()

# Obtenir le bucket
bucket = client.get_bucket(BUCKET_NAME)

# Télécharger le fichier
def download_file_from_bucket(bucket, file_path: str) -> str:
    blob = bucket.blob(file_path)
    local_file_name = os.path.basename(file_path)
    local_filepath = os.path.join(DOWNLOADED_LOCAL_DIRECTORY, local_file_name)
    blob.download_to_filename(local_filepath)
    print(f"Downloaded '{file_path}' to '{local_file_name}'")
    return local_filepath





In [8]:
def csv_to_documents(local_filepath: str) -> list[Document]:
    df = pd.read_csv(local_filepath)
    df['answer'] = df['answer'].fillna("Pas de réponse disponible")
    df['focus_area'] = df['focus_area'].fillna("Non catégorisé")
    documents = []
    
    for index, row in df.iterrows():
        # Content contient uniquement la question
        content = row['question']
        
        # Metadata contient la réponse et les informations contextuelles
        metadata = {
            "answer": row['answer'],
            "source": row['source'],
            "focus_area": row['focus_area'],
            "row_index": index
        }
        
        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)
    
    return documents

In [9]:
# Utilisation
file_path = "data/medquad.csv"  # Remplacez par votre nom de fichier
# local_filepath = download_file_from_bucket(bucket, file_path)

# Utiliser directement le fichier local
local_filepath = "./downloaded_files/medquad.csv"


documents = csv_to_documents(local_filepath)

In [10]:
# Vérification
print(f"Nombre de documents créés : {len(documents)}")
print("\nExemple du premier document :")
print(documents[0].page_content)
print("\nMétadonnées :", documents[0].metadata)

Nombre de documents créés : 16412

Exemple du premier document :
What is (are) Glaucoma ?

Métadonnées : {'answer': "Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic n

# Ingest data to an SQL Table


In [11]:
%pip install --upgrade --quiet  langchain-google-cloud-sql-pg langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
from dotenv import load_dotenv
load_dotenv()

True

In [37]:
# Configuration de la connexion
engine = PostgresEngine.from_instance(
    project_id="projet-gcp-450616",
    instance="soraka-instance",
    region="europe-west1",
    database="health_db",
    user="postgres",
    password=DB_PASSWORD  # Celui que vous avez défini pour votre instance
)

In [14]:
embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest",
    project="projet-gcp-450616"
)

In [15]:
import nest_asyncio
nest_asyncio.apply()

async def init_db():
    try:
        # Utiliser la table existante
        vector_store = PostgresVectorStore.create_sync(
            engine=engine,
            table_name="medical_qa",
            embedding_service=embeddings,
        )
        print("Vector store créé avec succès")
        return vector_store
    except Exception as e:
        print(f"Error: {e}")
        return None

# Création du vector store uniquement
vector_store = await init_db()

Vector store créé avec succès


In [16]:
'''
print(f"Nombre total de documents : {len(documents)}")

# Vérifier s'il y a des NaN dans les documents
nan_count = 0
for doc in documents:
    if 'NaN' in str(doc.metadata) or 'nan' in str(doc.metadata):
        nan_count += 1
print(f"\nNombre de documents contenant NaN : {nan_count}")

# Vérifier les remplacements
reponse_manquante = sum(1 for doc in documents if doc.metadata['answer'] == 'Pas de réponse disponible')
focus_non_cat = sum(1 for doc in documents if doc.metadata['focus_area'] == 'Non catégorisé')

print(f"\nNombre de 'Pas de réponse disponible' : {reponse_manquante}")
print(f"Nombre de 'Non catégorisé' : {focus_non_cat}")

# Afficher quelques exemples
print("\nExemples de documents avec valeurs remplacées :")
for doc in documents[:5]:
    print("\nQuestion:", doc.page_content)
    print("Métadonnées:", doc.metadata)
    '''

'\nprint(f"Nombre total de documents : {len(documents)}")\n\n# Vérifier s\'il y a des NaN dans les documents\nnan_count = 0\nfor doc in documents:\n    if \'NaN\' in str(doc.metadata) or \'nan\' in str(doc.metadata):\n        nan_count += 1\nprint(f"\nNombre de documents contenant NaN : {nan_count}")\n\n# Vérifier les remplacements\nreponse_manquante = sum(1 for doc in documents if doc.metadata[\'answer\'] == \'Pas de réponse disponible\')\nfocus_non_cat = sum(1 for doc in documents if doc.metadata[\'focus_area\'] == \'Non catégorisé\')\n\nprint(f"\nNombre de \'Pas de réponse disponible\' : {reponse_manquante}")\nprint(f"Nombre de \'Non catégorisé\' : {focus_non_cat}")\n\n# Afficher quelques exemples\nprint("\nExemples de documents avec valeurs remplacées :")\nfor doc in documents[:5]:\n    print("\nQuestion:", doc.page_content)\n    print("Métadonnées:", doc.metadata)\n    '

In [17]:
'''print(f"Nombre total de documents : {len(documents)}")

# Vérifier chaque champ séparément
nan_in_answer = 0
nan_in_source = 0
nan_in_focus = 0

for doc in documents:
    if pd.isna(doc.metadata['answer']) or doc.metadata['answer'] == 'NaN':
        nan_in_answer += 1
    if pd.isna(doc.metadata['source']) or doc.metadata['source'] == 'NaN':
        nan_in_source += 1
    if pd.isna(doc.metadata['focus_area']) or doc.metadata['focus_area'] == 'NaN':
        nan_in_focus += 1

print("\nAnalyse des NaN par champ :")
print(f"NaN dans answer : {nan_in_answer}")
print(f"NaN dans source : {nan_in_source}")
print(f"NaN dans focus_area : {nan_in_focus}")

print(f"\nValeurs remplacées :")
print(f"'Pas de réponse disponible' : {sum(1 for doc in documents if doc.metadata['answer'] == 'Pas de réponse disponible')}")
print(f"'Non catégorisé' : {sum(1 for doc in documents if doc.metadata['focus_area'] == 'Non catégorisé')}")

# Afficher un exemple de document avec valeur remplacée
print("\nExemple de document avec 'Pas de réponse disponible' ou 'Non catégorisé' :")
for doc in documents:
    if doc.metadata['answer'] == 'Pas de réponse disponible' or doc.metadata['focus_area'] == 'Non catégorisé':
        print("\nQuestion:", doc.page_content)
        print("Métadonnées:", doc.metadata)
        break'''

'print(f"Nombre total de documents : {len(documents)}")\n\n# Vérifier chaque champ séparément\nnan_in_answer = 0\nnan_in_source = 0\nnan_in_focus = 0\n\nfor doc in documents:\n    if pd.isna(doc.metadata[\'answer\']) or doc.metadata[\'answer\'] == \'NaN\':\n        nan_in_answer += 1\n    if pd.isna(doc.metadata[\'source\']) or doc.metadata[\'source\'] == \'NaN\':\n        nan_in_source += 1\n    if pd.isna(doc.metadata[\'focus_area\']) or doc.metadata[\'focus_area\'] == \'NaN\':\n        nan_in_focus += 1\n\nprint("\nAnalyse des NaN par champ :")\nprint(f"NaN dans answer : {nan_in_answer}")\nprint(f"NaN dans source : {nan_in_source}")\nprint(f"NaN dans focus_area : {nan_in_focus}")\n\nprint(f"\nValeurs remplacées :")\nprint(f"\'Pas de réponse disponible\' : {sum(1 for doc in documents if doc.metadata[\'answer\'] == \'Pas de réponse disponible\')}")\nprint(f"\'Non catégorisé\' : {sum(1 for doc in documents if doc.metadata[\'focus_area\'] == \'Non catégorisé\')}")\n\n# Afficher un exemp

In [ ]:
#premiere execution seulement pour remplir la table avec les documents

#vector_store.add_documents(documents)

['ce590b4f-4e4c-424a-826a-0c442ae14f99',
 'f59ff6db-b5b6-490d-84a3-e1bf90145d37',
 '6769268f-cc1c-4e4f-9a3c-71f118640af9',
 '4e628a99-0bd5-4e89-8f00-6e5541d98ff7',
 '6b72358a-f3b1-4028-8ebd-a69125fce6df',
 '28b2a736-70bd-4c63-b00b-c6a92b5cbaa9',
 '3d15a14b-ad26-46d2-be14-119b83f8d257',
 '72722850-d658-46ef-8b0a-d59ca8fd4ea9',
 '0c815ac0-0321-49bf-9702-fc0a44243bf7',
 '3e9a0ebe-8f38-4ee9-b112-ca1433be090d',
 'da29d74e-1aec-4640-a800-62670b04c538',
 'ff205f50-367d-4f34-b0bf-3a8198ec3424',
 '61fccb1b-12e9-4883-b9cd-228ce925e8ca',
 '3813b41f-11c6-4853-8c09-9f5b1dcc7b38',
 '0846c968-fb94-4499-8d33-56b9499261f2',
 '90b0e390-f4d4-4280-a8f1-3623e27b0f18',
 'e4c6a646-7e77-474e-a4c7-1315a01537fc',
 'c2e94986-e7b3-4658-b896-320774c53115',
 '49ee0606-15da-4ade-8e28-87e7647f2e47',
 '56d9dc84-5368-46b9-bd67-c9fedc6055ea',
 '7639a152-ffec-44b4-ab4c-96bdc63b0903',
 'd0325cb6-a1ad-4bb1-a761-8a5eacd068ca',
 '91fc2420-de98-4359-82f1-b73cc96ce058',
 'bd9ea29b-6221-450f-b769-52c77c3fbec5',
 'd909a824-77d9-

# Query search


In [61]:
query = "what is fever ?"

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)

docs = retriever.invoke(query)

In [63]:
results = vector_store.similarity_search_with_score(query, k=5)

print(f"Found {len(results)} documents\n")

for doc, score in results:
    print(f"Score: {score}")
    print("Question:", doc.page_content)
    print("Answer:", doc.metadata['answer'])
    print("Source:", doc.metadata['source'])
    #print("Focus Area:", doc.metadata['focus_area'])
    print("\n---\n")
    

Found 5 documents

Score: 0.12286249809115068
Question: What is (are) Fever ?
Answer: A fever is a body temperature that is higher than normal. It is not an illness. It is part of your body's defense against infection. Most bacteria and viruses that cause infections do well at the body's normal temperature (98.6 F). A slight fever can make it harder for them to survive. Fever also activates your body's immune system.    Infections cause most fevers. There can be many other causes, including       -  Medicines    -  Heat exhaustion    -  Cancers    -  Autoimmune diseases       Treatment depends on the cause of your fever. Your health care provider may recommend using over-the-counter medicines such as acetaminophen or ibuprofen to lower a very high fever. Adults can also take aspirin, but children with fevers should not take aspirin. It is also important to drink enough liquids to prevent dehydration.
Source: MPlusHealthTopics

---

Score: 0.2758027437151267
Question: What is (are) Q fe